In [1]:
import sys
from pathlib import Path

!uv pip install plotly
import plotly.express as px

sys.path.insert(0, str(Path().resolve().parent))
from processors import ad_risk

Using Python 3.12.3 environment at: /opt/venv
Resolved 3 packages in 100ms                                         
Prepared 2 packages in 385ms                                             
Installed 2 packages in 80ms                                
 + narwhals==2.8.0
 + plotly==6.3.1


In [2]:
# loading AD risk prediction model
adrisk = ad_risk.ADriskFromVCF()

Loading Seq2Reg model...
Loading Seq2Reg gene model...
Creating Seq2Gene model...
Model class: <class 'seq2gene.model_combined_modulator.Seq2GenePredictorCombinedModulator'>
Model architecture:
Model: Seq2GenePredictorCombinedModulator
  start_tkn: 96,768 params
  cre_tokenizer: 31,826,153 params
  gene_tokenizer: 31,826,153 params
  gene_map: 787,968 params
  cre_map: 787,968 params
  combined_modulator: 1,157,298,176 params
  tissue_heads: 4,726,273 params
  gene_loss: 0 params
Total number of parameters: 1,227,349,459
Loading checkpoint from /app/_artifacts/v4_pcg_epoch11_checkpoint.pth


Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Model loaded successfully on cuda


In [3]:
# picking a vcf file
vcf_path = "/app/_artifacts/HG00096.vcf.gz"

# picking a set of genes and their corresponding tissue ids to query
tissue_ids = [46, 47]
gene_ids = ["ENSG00000000457.13"] * len(tissue_ids)

In [4]:
preds = adrisk(vcf_path, gene_ids, tissue_ids)

Restoring states from the checkpoint path at /app/_artifacts/v4_pcg_epoch11_checkpoint.pth


Loaded BPE vocabulary from /app/vocabs/bpe_vocabulary_500.json
Filtered query df to 2 genes reducing from 2


/opt/venv/lib/python3.12/site-packages/lightning/pytorch/trainer/call.py:283: Be aware that when using `ckpt_path`, callbacks used to create the checkpoint need to be provided during `Trainer` instantiation. Please add the following callbacks: ["ModelCheckpoint{'monitor': None, 'mode': 'min', 'every_n_train_steps': 0, 'every_n_epochs': 1, 'train_time_interval': None}"].
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at /app/_artifacts/v4_pcg_epoch11_checkpoint.pth
2025-10-17 21:18:36 - utils.assets - INFO - Downloading from S3: s3://czi-dnacell-staging/model/common/cres_all_genes_manifest.parquet


Predicting: |                                                                                                 …

2025-10-17 21:18:36 - botocore.credentials - INFO - Found credentials from IAM Role: vcp-dev-scidata
2025-10-17 21:18:37 - utils.assets - INFO - Loading parquet file: /tmp/tmpisgkiw47/model/common/cres_all_genes_manifest.parquet
2025-10-17 21:18:37 - utils.assets - INFO - Validated schema - found columns: {'gene_id', 'file_path'}
2025-10-17 21:18:37 - utils.assets - INFO - Downloading from S3: s3://czi-dnacell-staging/model/common/cres_all_genes/ENSG00000000457.13/gene_vocab.csv
Running AD predictor: 0it [00:00, ?it/s]2025-10-17 21:18:48 - utils.assets - INFO - Downloading from S3: s3://czi-dnacell-staging/alzhimers_disease/v1/manifest.parquet
2025-10-17 21:18:48 - botocore.credentials - INFO - Found credentials from IAM Role: vcp-dev-scidata
2025-10-17 21:18:50 - utils.assets - INFO - Loading parquet file: /tmp/tmp6hggiea3/alzhimers_disease/v1/manifest.parquet
2025-10-17 21:18:50 - utils.assets - INFO - Validated schema - found columns: {'tissue_id', 'gene_id', 'file_path'}
2025-10-17

In [5]:
fig = px.bar(
    preds, x="tissue_name", y="ad_risk", title="AD Risk Predictions across Tissues"
)
fig.show()

In [6]:
preds

,gene_id,tissue_id,tissue_name,predicted_expression,embedding,ad_risk,gene_name
0,ENSG00000000457.13,46,minor salivary gland,2.223876,"[[0.10644531, 0.3203125, -1.8203125, 0.2636718...",0.627235,SCYL3
1,ENSG00000000457.13,47,muscle - skeletal,1.514095,"[[-2.21875, -1.625, -2.53125, -1.859375, 2.859...",0.666648,SCYL3
